# Generalizations of Max-Flow

The purpose of this assignment is to learn about the min-cost flow problem, a generalization of max-flow, and to familiarize yourself with implementing and solving linear programs.

## Min-Cost Flow

Recall that a flow network with demands consists of a directed graph $G = (V, E)$, where each edge $(i,j) \in E$ has a positive integer capacity $c_{ij}$ and each node $i \in V$ has an integer demand $d_i$. In a *min-cost flow* problem, each edge $(i,j) \in E$ also has a cost (or weight) $w_{ij}$. (Note that this input generalizes the input to two important problems we discussed so far: in max flow, the edge weights were not important while in shortest paths, the edge capacities were not important.) 

Given a flow network with capacities and costs, the goal is to find a *feasible* flow $f: E \rightarrow R^+$ --that is, a flow satisfying edge capacity constraints and node demands-- that minimizes the total cost of the flow. Explicitly, the problem can be formulated as a linear program.

### Question 1

Answer Problem 1 in HW4-theoretical.

### Question 2

To implement your reduction from Problem 1 in HW4-theoretical, you will work with some standard benchmark instances for min-cost flow found [here](http://elib.zib.de/pub/Packages/mp-testdata/mincost/gte/index.html). The format of the data is described in the [Info](http://elib.zib.de/pub/Packages/mp-testdata/mincost/gte/info) file. You are to read in the graph from the data file in a form that can be solved using NetworkX's `min_cost_flow` function. Note that the data sometimes lists multiple edges between the same nodes, but with different costs or capacities. In forming the graph, you need to implement your reduction from the previous question and form a `DiGraph` instance, because the `min_cost_flow` function cannot handle multi-edges, even though the package offers `MultiDiGraph` objects.

In [1]:
import networkx as nx

def create_graph(infile):
    myfile=open(infile,'r')
    x=0
    G = nx.Graph()
    G = nx.DiGraph(G)     # make graph directed
    list1=[]
    list2=[]
    while True:
        line=myfile.readline().strip()#read file line by line
        if line.split(' ')[0]=='n':
            t=str(line.split(' ')[1])
            list2.append(t)
            G.add_node(t)
            G.node[t]['demand'] = int(line.split(' ')[2]) #add demand attribute to G
        if line.split(' ')[0]=='a':
            b=str(line.split(' ')[1])
            c=str(line.split(' ')[2])
            if (b,c) in list1:             #tell if there are multiedges between b and c
                x=x+1 
                G.add_node(x)              #if yes, add a node x
                G.node[x]['demand'] = 0
                G.add_edge(b,x)
                G.add_edge(x,c)
                G.edge[b][x]['capacity'] = int(line.split(' ')[4]) #make the capacity equals original edge
                G.edge[x][c]['capacity'] = int(line.split(' ')[4])
                G.edge[b][x]['weight']=int(line.split(' ')[5])/2.0 # change weight to half of original number
                G.edge[x][c]['weight']=int(line.split(' ')[5])/2.0
                G.edge[b][x]['lower_bound'] = int(line.split(' ')[3]) #lower bound keep unchanged
                G.edge[x][c]['lower_bound'] = int(line.split(' ')[3])
            else:
                G.add_node(b)
                G.add_node(c)
                if b not in list2:
                    G.node[b]['demand'] = 0      # add kinds of attribute to graph
                if c not in list2:
                    G.node[c]['demand'] = 0
                G.add_edge(b,c)
                G.edge[b][c]['capacity'] = int(line.split(' ')[4])
                G.edge[b][c]['weight']=int(line.split(' ')[5])
                G.edge[b][c]['lower_bound']=int(line.split(' ')[3])
                list1.append((b,c))
        if line.split(' ')[0]=='':       # if we are in the blank line, then jump out
            break 
    return G

The following will check that your code outputs the expected min cost flow values on several test instances.

In [2]:
G_40 = create_graph('/Users/Alexander/Desktop/gte_bad.40.txt')
G_6830 = create_graph('/Users/Alexander/Desktop/gte_bad.6830.txt')
G_176280 = create_graph('/Users/Alexander/Desktop/gte_bad.176280.txt')

print "Correct value for _40 instance:", nx.min_cost_flow_cost(G_40) == 52099553858
print "Correct value for _6830 instance:", nx.min_cost_flow_cost(G_6830) == 299390431788
print "Correct value for _176280 instance:", nx.min_cost_flow_cost(G_176280) == 510585093810

Correct value for _40 instance: True
Correct value for _6830 instance: True
Correct value for _176280 instance: True


## Linear Programming

Instead of using special-purpose min-cost flow solvers, you will now formulate the problems as linear programs and use general-purpose LP solvers to find the solutions.

### Question 3

Implement the following function to formulate the flow LP and return the optimal value (i.e., minimum cost over feasible flows).

In [3]:
import pulp

def lp_flow_value(G):
    nodelist=[]
    for a in G.nodes():
        nodelist.append(a) #store all nodes
    ndata={}
    for b in nodelist:
        ndata[b]=G.node[b]['demand']   #store demand of every node
    arclist=[]
    for c in G.edges():  #store every edge
        arclist.append(c)
    arcdata={}
    for d in arclist:     #store attributes of every edge
        x,y=d
        arcdata[d]=[G.edge[x][y]['weight'],G.edge[x][y]['lower_bound'],G.edge[x][y]['capacity']]
    vars = pulp.LpVariable.dicts("Route",arclist,None,None) #creates Variables
    for m in arclist:
        vars[m].bounds(arcdata[m][1], arcdata[m][2])  #add the upper and lower bounds of variables
    prob = pulp.LpProblem("min_cost",pulp.LpMinimize) #add the 'prob' variable to store the problem data
    prob+= pulp.lpSum([vars[m]* arcdata[m][0] for m in arclist]) #build the objective function
    for n in nodelist:
        prob+= (pulp.lpSum([vars[(i,j)] for (i,j) in arclist if i == n]) ==
        ndata[n]+ pulp.lpSum([vars[(i,j)] for (i,j) in arclist if j == n])) #give constraints for the problem
    
    prob.writeLP("min_cost.lp") #write problem data to file
    prob.solve()    #solve problem
    return int(pulp.value(prob.objective)) 

The following will check that the LP finds the same optimal values as previously.

In [4]:
print "Correct value for _40 instance:", lp_flow_value(G_40) == 52099553858
print "Correct value for _6830 instance:", lp_flow_value(G_6830) == 299390431788
print "Correct value for _176280 instance:", lp_flow_value(G_176280) == 510585093810

Correct value for _40 instance: True
Correct value for _6830 instance: True
Correct value for _176280 instance: True
